<a href="https://colab.research.google.com/github/nephelim74/BeginProg/blob/main/spark_dz2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [15]:
# Создаем SparkSession
spark = SparkSession.builder \
    .appName("Employee Salary Comparison") \
    .getOrCreate()

In [16]:
# Данные
data = [
    (1, 'Joe', '70', '3'),
    (2, 'Henry', '80', '4'),
    (3, 'Sam', '60', None),
    (4, 'Max', '90', None)
]


In [17]:
# Создаем DataFrame с явным указанием схемы
columns = ['id', 'Name', 'Salary', 'ManagerId']
df = spark.createDataFrame(data, schema=columns)

In [18]:
# Преобразуем Salary в числовой формат
df = df.withColumn("Salary", df["Salary"].cast("int"))

In [19]:
# Печатаем исходный DataFrame
print("Исходный DataFrame:")
df.show()

Исходный DataFrame:
+---+-----+------+---------+
| id| Name|Salary|ManagerId|
+---+-----+------+---------+
|  1|  Joe|    70|        3|
|  2|Henry|    80|        4|
|  3|  Sam|    60|     NULL|
|  4|  Max|    90|     NULL|
+---+-----+------+---------+



In [20]:
# Объединяем таблицу с самой собой для сопоставления сотрудников и их менеджеров
joined_df = df.alias("employee") \
    .join(df.alias("manager"), F.col("employee.ManagerId") == F.col("manager.id"), "left") \
    .select(F.col("employee.Name").alias("EmployeeName"),
            F.col("employee.Salary").alias("EmployeeSalary"),
            F.col("manager.Name").alias("ManagerName"),
            F.col("manager.Salary").alias("ManagerSalary"))

In [21]:
# Фильтруем сотрудников, которые зарабатывают больше своих менеджеров
result_df = joined_df.filter(F.col("EmployeeSalary") > F.col("ManagerSalary")) \
    .select("EmployeeName")

In [22]:
# Печатаем результат
print("Сотрудники, которые зарабатывают больше своих менеджеров:")
result_df.show()

# Останавливаем SparkSession
spark.stop()

Сотрудники, которые зарабатывают больше своих менеджеров:
+------------+
|EmployeeName|
+------------+
|         Joe|
+------------+

